In [79]:
from Tools.scripts.generate_opcode_h import header
with open('cred/Client_id.txt.gitignore', 'r') as f:
CLIENT_ID = f.read()
with open('cred/Key.txt.gitignore', 'r') as f:
SECRET_KEY = f.read()

In [80]:
import requests

In [81]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [82]:
with open('cred/pw.txt.gitignore', 'r') as f:
    pw = f.read()
with open('cred/un.txt.gitignore', 'r') as f:
    un = f.read()

In [83]:
data = {
    'duration' : 'permanent',
    'grant_type': 'password',
    'username': un,
    'password': pw,
}

In [84]:
headers = {'User-Agent': 'MyAPI/0.0.1'}

In [85]:
res = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth=auth,
    data=data,
    headers=headers
)

In [86]:
if res.status_code != 200:
    print("Authentication failed!", res.text)
else:
    print("Authenticated successfully!")

Authenticated successfully!


In [87]:
TOKEN = res.json()['access_token']

In [88]:
headers['Authorization'] = f'Bearer {TOKEN}'

In [89]:
response = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
print(response.status_code)
print(response.json())


200
{'is_employee': False, 'seen_layout_switch': False, 'has_visited_new_profile': False, 'pref_no_profanity': True, 'has_external_account': False, 'pref_geopopular': 'GLOBAL', 'seen_redesign_modal': False, 'pref_show_trending': True, 'subreddit': {'default_set': True, 'user_is_contributor': False, 'banner_img': '', 'restrict_posting': True, 'user_is_banned': False, 'free_form_reports': True, 'community_icon': None, 'show_media': True, 'icon_color': '', 'user_is_muted': None, 'display_name': 'u_Ourbatshitt', 'header_img': None, 'title': '', 'coins': 0, 'previous_names': [], 'over_18': False, 'icon_size': [256, 256], 'primary_color': '', 'icon_img': 'https://styles.redditmedia.com/t5_5qzimb/styles/profileIcon_snooe81a9980-71d8-4876-a728-302668acde10-headshot.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=6a532fdfb6ea337b1f9196908d6a24f482937ea4', 'description': '', 'allowed_media_in_comments': [], 'submit_link_label': '', 'header_size': None, 'restrict_commenting': False, 'su

In [90]:
res = requests.get('https://oauth.reddit.com/r/MogelPackung/hot',
                 headers=headers)

In [91]:
for post in res.json()['data']['children']:
    print(post['data']['title'])

Haribo
Die haben in der Fabrik aus meiner Cola gesoffen!
Jetzt auch bei Gemüsebrühe: 20% weniger Inhalt!
Keine Schokolade?!?
Jetzt reicht's aber hier
Mogelpackung bei der Schanzenbäckerei - Knusperhähnchen-Schnitzelbrötchen nur noch halb so gut? 😡
Zählt das?
Homann weniger Inhalt
Hobnob Flapjacks
Ist das schon Diebstahl?
Räucherlachs 93g statt 100g
Süßstoff verdünnt?
ja! Wiener Würstchen
Rewe - "näher geht's nicht, frischer geht's nicht" Werbung
MAOAM fängt auch damit an
PU-Schaum
Aso
50% Luft
Bemerkt bei mir auf der Arbeit, zählt es?
Ich ruf gleich den Verbraucherschutz
Kirchliche Mogelpackung
Positiv: Zu viel Volumen muss nicht sein
Zählt das?
4,7 Stücken Wurst. Die Sauce in einem Stück auf dem anderen Ende der "Pizza".
Zwei Mogeleien auf einen Streich


In [98]:
import pandas as pd

In [99]:
df = pd.DataFrame()

In [108]:
posts = []

for post in res.json()['data']['children']:
    data = post['data']
    posts.append({
        'title': data['title'],
        'subreddit': data['subreddit'],
        'score': data['score'],
        'upvote_ratio': data['upvote_ratio'],
        'author': data['author'],
        'num_comments': data['num_comments'],
        'created_utc': data['created_utc'],
        'selftext': data['selftext'],
        'permalink': f"https://reddit.com{data['permalink']}"
    })
df = pd.DataFrame(posts)
df.head()
df.to_csv("data/raw/reddit_posts.csv", index=False)

,title,subreddit,score,upvote_ratio,author,num_comments,created_utc,selftext,permalink
0,Haribo,MogelPackung,89,0.92,Agreeable-Slip-884,26,1.750872e+09,Von 37 cm Verpackung sind 54 % Luft. Toll star...,https://reddit.com/r/MogelPackung/comments/1lk...
1,Die haben in der Fabrik aus meiner Cola gesoffen!,MogelPackung,27,0.74,Sweet-Highlight3205,25,1.750879e+09,,https://reddit.com/r/MogelPackung/comments/1lk...
2,Jetzt auch bei Gemüsebrühe: 20% weniger Inhalt!,MogelPackung,0,0.30,Rhodianer,6,1.750941e+09,"Gerade aufgemacht, aber ein Fabrikmitarbeiter ...",https://reddit.com/r/MogelPackung/comments/1lk...
3,Keine Schokolade?!?,MogelPackung,0,0.41,Far_Squirrel_6148,18,1.750913e+09,"Meine erste Begegnung mit ChoViva, der kakaofr...",https://reddit.com/r/MogelPackung/comments/1lk...
4,Jetzt reicht's aber hier,MogelPackung,1587,0.98,Luis_9466,79,1.750673e+09,,https://reddit.com/r/MogelPackung/comments/1li...


In [109]:
import time

all_comments = []

for post in posts:
    post_id = post['permalink'].split('/')[-3]  # Extract post ID from permalink
    
    url = f"https://oauth.reddit.com/comments/{post_id}"
    response = requests.get(url, headers=headers, params={"limit": 20})  # limit number of comments
    time.sleep(1)  # Respect API rate limits
    
    if response.status_code == 200:
        json_response = response.json()
        comments = json_response[1]['data']['children']

        for comment in comments:
            if comment['kind'] == 't1':  # t1 = comment
                data = comment['data']
                all_comments.append({
                    'post_title': post['title'],
                    'post_subreddit': post['subreddit'],
                    'comment_body': data.get('body', ''),
                    'comment_author': data.get('author', ''),
                    'score': data.get('score', 0),
                    'created_utc': data.get('created_utc', 0),
                    'post_permalink': post['permalink'],
                })
df_comments = pd.DataFrame(all_comments)
df_comments.head()
df_comments.to_csv("data/raw/reddit_comments.csv", index=False)

KeyboardInterrupt: 

In [110]:
pip install praw

  Obtaining dependency information for praw from https://files.pythonhosted.org/packages/73/ca/60ec131c3b43bff58261167045778b2509b83922ce8f935ac89d871bd3ea/praw-7.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for prawcore<3,>=2.4 from https://files.pythonhosted.org/packages/96/5c/8af904314e42d5401afcfaff69940dc448e974f80f7aa39b241a4fbf0cf1/prawcore-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for update_checker>=0.18 from https://files.pythonhosted.org/packages/0c/ba/8dd7fa5f0b1c6a8ac62f8f57f7e794160c1f86f31c6d0fb00f582372a3e4/update_checker-0.18.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/189.3 kB ? eta -:--:--
   ----------------- ---------------------- 81.9/189.3 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 189.3/189.3 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: D:\pycharm projekt für unternehmenssoftware\Unternehmenssoftware\.venv\Scripts\python.exe -m pip install --upgrade pip


In [112]:
import pandas as pd
import os

input_csv = "C:/Users/Laura/Downloads/2025-06-26_schleswig-holstein.csv"
output_dir = "data/raw"
output_csv = os.path.join(output_dir, "brands.csv")

df = pd.read_csv(input_csv)
brands = (
    df["brand"]
    .dropna()                                 
    .astype(str)
    .str.strip()
    .loc[lambda x: x.str.upper() != "NA"]     # Remove "NA"
    .str.lower()
    .unique()
    .tolist()
)
os.makedirs(output_dir, exist_ok=True)
pd.DataFrame({"brand": brands}).to_csv(output_csv, index=False)

print(f"✅ Extracted {len(brands)} unique brand(s) to {output_csv}")

✅ Extracted 1341 unique brand(s) to data/raw\brands.csv


In [115]:
from transformers import pipeline

# Load the classification pipeline with the specified model
pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

# Classify a new sentence
sentence = "I love this product! It's amazing and works perfectly."
result = pipe(sentence)

# Print the result
print(result)


NameError: name 'torch' is not defined